The embedded structure has been chosen over the referencing structure. The embedded structure discourages data redundancy as no referring fields are required to be added as foreign keys to show relationships. The referencing structure, however, do require foreign keys to show relationships. The drawbacks of the embedded structure may be its troublesome to read or process. However, these are quite simple solutions through the pretty print method allowing for us to easily format and understand the structure, and the unwind method to easily flatte then the data structure so processing and aggregation can occur.

An example of the embedded model without no hotspots is as follows:

<code> {'GHI_w/m2': 156,
 '_id': ObjectId('60abb6caf7b620cbe44e88d0'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2018, 12, 12, 0, 0),
 'max_wind_speed': 12.0,
 'precipitation ': 0.0,
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}</code>

As you can see from the above document, there isn't any embedded fields which is due to the fact that there are no hotspots

Below is an exmaple of the embedded model with hotspots.

<code>{'GHI_w/m2': 159,
 '_id': ObjectId('60abb6caf7b620cbe44e87c9'),
 'air_temperature_celcius': 20,
 'average_surface_temp': 49.0,
 'date': datetime.datetime(2018, 3, 24, 0, 0),
 'hotspot_location': [{'confidence': 65,
                       'date': datetime.datetime(2018, 3, 24, 0, 0),
                       'datetime': '2018-03-24T04:58:00',
                       'latitude': -37.1603,
                       'longitude': 142.8088,
                       'surface_temperature_celcius': 50},
                      {'confidence': 55,
                       'date': datetime.datetime(2018, 3, 24, 0, 0),
                       'datetime': '2018-03-24T04:56:10',
                       'latitude': -37.1684,
                       'longitude': 142.8183,
                       'surface_temperature_celcius': 48}],
 'max_wind_speed': 11.1,
 'precipitation ': 0.0,
 'relative_humidity': 58.4,
 'report': 'I',
 'station': 948701,
 'windspeed_knots': 6.1} </code>
 
 As we can see that there are two hotspots as the field 'hotspot_location' contains two elements in its array.
 This is the embedded structure or informally / colloqiually known as the nested structure. Not to mention, this format provides is the most space efficient format as no extra fields are required to be added to the structure. 

In [ ]:
# importing the required dependencies
import csv
import pymongo
from pymongo import MongoClient
from pprint import pprint
client = MongoClient()
import pandas

In [ ]:
# defining variables which will be used for pymongo commands
db = client.fit3182_assignment_db
climate_historic = db.climate_historic
unwound = db.unwound
q2f = db.q2f

In [ ]:
# Dropping existing databases before creating the new database
climate_historic.drop()
unwound.drop()
q2f.drop()
embedded.drop()

In [ ]:
with open('climate_historic.csv') as csv_file: # Opening csv file
    csv_reader = csv.DictReader(csv_file) # Using DictReader makes into Json Files
    for row in csv_reader: # Goes through each Json record and ensures correct data types as all was initialised as strings
        row['date'] = pandas.to_datetime(row['date'],dayfirst=True)
        row['station'] = int(row['station'])
        row['air_temperature_celcius'] = int(row['air_temperature_celcius'])
        row['relative_humidity'] = float(row['relative_humidity'])
        row['windspeed_knots'] = float(row['windspeed_knots'])
        row['max_wind_speed'] = float(row['max_wind_speed'])
        row['GHI_w/m2'] = int(row['GHI_w/m2'])
        row['report'] = str(row['precipitation '][-1]) # Separate precipitation to make apppropriate data type
        row['precipitation '] = float(row['precipitation '][1:-2])
        row['average_surface_temp'] = 'N/A' # THis is needed to compute the average surface temperature later on
        row['hotspot_location'] = [] # Needed for later aggregation
        climate_historic.insert_one(row) #Insert Json record to database
        
        
        
        

In [ ]:
with open('hotspot_historic.csv') as csv_file:# Opening csv file
    csv_reader = csv.DictReader(csv_file) # Using DictReader makes into Json Files
    for row in csv_reader: # Goes through each Json record and ensures correct data types as all was initialised as strings
        row['latitude'] = float(row['latitude'])
        row['longitude'] = float(row['longitude'])
        row['confidence'] = int(row['confidence'])
        row['date'] = pandas.to_datetime(row['date'],dayfirst=True)
        row['surface_temperature_celcius'] = int(row['surface_temperature_celcius'])
        climate_historic.update_one({'date':row['date']},{'$push':{'hotspot_location':row}}) # We match by date and push the entire document to the climate record
        x= climate_historic.find_one({'date':row['date']})['average_surface_temp'] # Calculation of average
        if x == 'N/A':
            new_average = row['surface_temperature_celcius'] # In the case of no hotspot embedded yet, change N/A to hotspot surface temp
        else:
            n=[i for i in climate_historic.aggregate([{'$match':{'date':row['date']}},
                                           {'$project':{'no_of_fires':{'$size':'$hotspot_location'}}}])][0]['no_of_fires']#Finds number of fires in one date by grabbing length of list
            new_average = x*(n-1) # Multiply by number of fires -1
            new_average+=row['surface_temperature_celcius'] # Add new fire
            new_average/= n # Divide by number of fires for new average
            
        climate_historic.update_one({'date':row['date']},{'$set':{'average_surface_temp': new_average}}) # Update the new average


In [ ]:
#Note: unwond only has documents that have hotspots
# Creating new collection called unwound which unwinds hotsot data for future use
res = climate_historic.aggregate([{'$unwind':'$hotspot_location'}])
i = 0
for doc in res:
    doc['_id'] = i #Creating unique index
    unwound.insert_one(doc) #Inserting doc
    i+=1

In [ ]:
x = climate_historic.find()
for doc in x:
    pprint(doc)

In [ ]:
res = climate_historic.find({"date": pandas.to_datetime("12/12/2018")},{"air_temperature_celcius":1, "date":1, "max_wind_speed":1, "precipitation ":1, "relative_humidity":1, "station":1 , "windspeed_knots":1, "GHI_w/m2":1}) # Finding climate data on 12/12/2018 with projections

In [ ]:
for doc in res:
    pprint(doc)

In [ ]:
long = 'hotspot_location.surface_temperature_celcius'

In [ ]:
unwound.find({'$and':[{long:{'$gte':65}}, {long:{'$lte':100}}]}).count() # finding the hotspot data with surfae temp betwen 65 and 100

In [ ]:
res = unwound.find({'$and':[{long:{'$gte':65}}, {long:{'$lte':100}}]},{"hotspot_location.latitude":1, "hotspot_location.longitude":1, "hotspot_location.surface_temperature_celcius":1, "hotspot_location.confidence":1})

In [ ]:
for doc in res:
    pprint(doc)

In [ ]:
x = unwound.find({"$or":[{"date": pandas.to_datetime("15/12/2018")},{"date":pandas.to_datetime("16/12/2018")}]},{"date":1, "hotspot_location.surface_temperature_celcius":1, "air_temperature_celcius":1, "relative_humidity":1, "max_wind_speed":1}) #finding data for the 15th and 16th December with correct projections
for d in x:
    pprint(d)

In [ ]:
long = "hotspot_location.confidence"

In [ ]:
x = unwound.find({'$and':[{long:{'$gte':80}}, {long:{'$lte':100}}]},{"hotspot_location.datetime":1, "air_temperature_celcius":1, "hotspot_location.surface_temperature_celcius":1, "hotspot_location.confidence":1}) #finding records where confidence is between 80 and 100 with correct projections
for d in x:
    pprint(d)

In [ ]:
results = unwound.find({}).sort("hotspot_location.surface_temperature_celcius", -1).limit(10) #sorting on surface temperature in descending order and using .limit for top 10 records
for result in results:
    pprint(result)

In [ ]:
results = climate_historic.aggregate([{"$project":{"_id":0,"date":1,"fires":{"$size":"$hotspot_location"}}}]) #number of fires based on the size of the hotspot_location array 
for document in results: 
    pprint(document) 

In [ ]:
embedded.drop()
q2f.drop()

In [ ]:
long = "hotspot_location.confidence"

In [ ]:
x = unwound.find({long:{'$lt':70}}) # finding records with confidence below 70
for d in x:
    pprint(d)

In [ ]:
results = climate_historic.aggregate([{"$project":{"_id":0,"date":1,"average_surface_temp":1}}]) #avg surface temp has already been calculated and is simply projected
for document in results: 
    pprint(document) 

In [ ]:
y = climate_historic.find().sort("GHI_w/m2").limit(10) #Using limit to see top 10 records

In [ ]:
for d in y:
    pprint(d)

In [ ]:
x = climate_historic.find({'$and':[{'report': 'G'},{'precipitation ': {'$gte':0.20}},{'precipitation ':{'$lte': 0.35}}]}) # Using and to fulfil multiple conditions including report, prefixed I

In [ ]:
for d in x:
    pprint(d)